Nom Etudiant 1: Assia MASTOR
Nom Etudiant 2: Maimouna ABDERAHMAN

# TME 2 : Median String


## Partie A : Recheche de pattern (motifs) en permettant des variations

Les motifs que nous cherchons dans les sequences d'ADN peuvent contenir quelques variations ou mutations. Nous allons developpé dans cet partie differents algoritmes pour la recherche de motifs variables. 

1\. Comme dans le TME precedent, nous allons d'abord générer des données atificielles pour pouvoir tester les algorithmes. Faire une fonction pour générer `t` séquences artificielles de taille `n`. Implantez dans chaque séquence un motif de taille `k` à des positions aléatoires avec `v` substitutions choisies aléatoirement.


In [53]:
import random

nuc = ('A', 'C', 'G', 'T')

k=4 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence

def insertMotif(sequence, motif, position):
    return sequence[:position] + motif + sequence[position:]

def generateRandomSequence(n:int, upper=False):
    """
    Génère une séquence nucléotidique aléatoire 
    entrée n : longueur de la sequence
    entrée upper : bool, si True les nucléotides seront minuscule, False majuscule
    sortie sequence : une séquence nucléotidique aléatoire 
    """
    sequence = ""
    
    for i in range (n):
        tmp=random.choice(nuc)
        sequence =sequence+tmp
        
    if upper== True:
        sequence= sequence.lower()
    return sequence

def modifierMotif(motif:str, nbpos:list,  upper=False):
    """
    Modifie nbpos positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée nbpos: nombre de positions
    entrée upper : bool, si True les nucléotides modifiés seront minuscule, False majuscule
    sortie motifM: motif modifié
    """
    
    motifM = list(motif)
    tmp=''
    randind=0
    up=upper
    
    for pos in range(nbpos[0]):
        if pos < len(motifM)-1:
            #Tirer un caractere aléatoire tmp parmi atgc et l'insérer à la position randind choisie aléatoirement 
            tmp = generateRandomSequence(1, up)
            randind= random.randint(0, len(motif)-1)
            motifM[randind]=tmp 
        
    return "".join(motifM)


def implantMotifVar(k, v, t, n): 
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée k: taille du motif
    entrée v: nombre de variations
    entrée t : nombre de séquences 
    entrée n : longueur des séquences
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus motif doit être égal à t, pensez à générer de séquence aléatoire de taille t-k pour pouvoir implanter un motif de taille k
    """
    
    sequences = []
    motifRand=generateRandomSequence(k,True)
    mofitfRandVariation=''
    seqTPM=''
    tailleSeqSanMotifImplanté = n-k
    positionRand=random.randint(0,tailleSeqSanMotifImplanté)
    
    for sep in range(t): #Création des séquences avec le motif générer qui varie de v base
        seqTPM=generateRandomSequence(tailleSeqSanMotifImplanté,False) #Sequence ne majuscule
        mofitfRandVariation= modifierMotif (motifRand,[v],True)        #Motif en minuscule
        
        #Insertion du motif dans la seq
        seqTPM= insertMotif(seqTPM, mofitfRandVariation, positionRand)
        sequences.append(list(seqTPM))#Ajout à la matrice
        positionRand=random.randint(0,tailleSeqSanMotifImplanté) 
        
    return sequences

#tester modifMotif
print (modifierMotif("acg", [2], False))

adn = implantMotifVar(k, v, t, n)
print (adn)



acG
[['C', 'A', 'T', 't', 'a', 't', 'g', 'T', 'T', 'C'], ['T', 't', 'a', 'g', 'g', 'C', 'A', 'G', 'A', 'T'], ['T', 'C', 'G', 'T', 'C', 'G', 't', 'a', 'g', 'g']]


2\. Visualisation de motifs. Nous pouvons visualiser les motifs à l'aide des outils de LOGO https://weblogo.berkeley.edu/logo.cgi. Executer votre fonction de generation de motif variable, extraire les motifs et visulise-le à l'aide de webLogo. Utiliser les parametre ci-dessous.


In [72]:
k=8 #taille de motif
v=2 #nb de positions variable dans le motif
t=10 #nb de sequences
n=10 #longuer des sequence

adn = implantMotifVar(k, v, t, n)
print (adn)


[['T', 'g', 'a', 't', 't', 'a', 'a', 'g', 'c', 'G'], ['g', 'a', 't', 't', 'a', 'c', 'g', 'c', 'C', 'C'], ['g', 't', 't', 't', 't', 't', 'g', 'c', 'G', 'G'], ['g', 'a', 't', 't', 'a', 't', 'a', 'c', 'C', 'T'], ['c', 'a', 't', 't', 'a', 't', 't', 'c', 'A', 'C'], ['C', 'T', 'g', 'a', 't', 't', 'a', 't', 'c', 'c'], ['G', 't', 'a', 't', 't', 'a', 'c', 'g', 'c', 'G'], ['C', 'G', 'c', 'a', 't', 't', 'a', 't', 'g', 'g'], ['g', 'a', 't', 't', 'a', 't', 'g', 'g', 'G', 'T'], ['g', 'a', 't', 'a', 'a', 't', 'g', 'c', 'G', 'A']]


3\. Implémentez l'algorithme _"Median String Search"_ pour chercher des motifs de taille variable. Tester le sur les jeux de données gnerer precedement. Vous devez éliminer les motifs peu complexe pour éviter les calculs inutiles. Pensez à eliminer tous les motifs ayant une formation diferents de motifs attendues, par exemple, les motifs ayant deux base repeté comme ACACACAC, etc.  

In [73]:
from itertools import product

def kmerList(allkmers):
    kmers=[]
    for v in allkmers:
        kmers.append("".join(v))
    return kmers 

#Genere tous les K-mers de taille K ayant de AAA... à TTT...
allkmers = product(nuc, repeat=k)
kmers = kmerList(allkmers)
#print(kmers) 
#print (len(kmers))

#Fonction qu'on utilise pour removeLowComplexe
def islowComplexe(motif, nb):
    """
    Renvoie un boolean pr dire si un motif est peu complex 
    entrée motifs: motif
    sortie motifsClean: boolean
    """
    
    SetDesCaractèresComposantsLeMotif = set(motif) #Suppression des doublons du motifs afin d'obtenir un set avec que les différents caractères qui le compose
    lenSet = len(SetDesCaractèresComposantsLeMotif)
    compteur=0
    
    if lenSet > 2:#si le motif k admet moins de 3 caractères différents il est ignoré car forcément 2 même lettres répétées ou la même lettre est répétée plusieurs fois à la suite
        #Test pour savoir si un mm caractère du motif n'est pas présent plus de nb fois à la suite
        for car in SetDesCaractèresComposantsLeMotif :
            mot=''
            for i in range (nb):
                mot=mot+car

            for y in range (len(motif)-nb+1): #On fait glisser une fenètre glissante pour le motif considérer tel que dés lors qu'on croise le mot de taille nb composé de la mm base, alors le motif est peu complex
                tmp=motif[y:y+nb]
                if mot==tmp:
                    return True
            
            compteur=compteur+1
            if compteur == lenSet:
                return False         
            
    return True


def removeLowComplexe(motifs,m):
    
    validMotif = []
    for motif in motifs:
        if not(islowComplexe (motif,m)):
            validMotif.append(motif)

    return validMotif


print (len(kmers))
kmersValid = removeLowComplexe(kmers, 3) #Nous supprimons tt séquences composées de la même base ou d'uniquement 2 bases, ainsi que tt séquences composé d'un sous-motif de 3 bases identiques qui se suivent à la suite
print (len(kmersValid))
#print(kmersValid)

65536
46680


In [74]:
def hamdist(str1:str, str2:str):
    """
    Calcul la distance de hamming entre deux chaînes de caractères
    entrée str1: chaîne de caractères
    entrée str2: chaîne de caractères
    sortie distance: distance de hamming
    """
    diffs = 0

    #str1 et str2 doivent etre de memme taille donc on le garantit en les mettant en majuscule
    strtmp1=str1.upper()
    strtmp2=str2.upper()
    
    #Détermination des différences
    if len(strtmp1)== len(strtmp2):
        for i in range (len(strtmp1)):
            if strtmp1[i]!= strtmp2[i]:
                diffs=diffs+1
                
    return diffs

def totalDistance(motif:str, sequences, k):
    """
    Calcul la totalDistance
    entrée motif: sequence consensus
    entrée sequences: chaîne de caractères
    entrée k: taille du motif 
    entrée sequences: matrice de dimension txn contenant les séquences
    sortie totalDistance: somme de distance de hamming minimal
    """
    td = 0
    colonneM= len(sequences[0])
    ligneN = len(sequences)
    
    for l in range(ligneN): #Parcours ligne de la matrice cad chaque séquence une à une
        tmpstr = ''.join(sequences[l])
        mincour= hamdist(motif, tmpstr[0:k])
        
        for i in range (len(tmpstr)-k+1):#Boucle sur la séquence pour déterminer distance de haming la plus petite 
            if(i==0):
                continue
            mintmp= hamdist(motif, tmpstr[i:i+k])
            mincour=min(mincour,mintmp)
        td = td + mincour
    
    return td


#TEST : HAMDIST ET TOTALDISTANCE

#strr1='holla'
#str2= 'hello'
#res = hamdist(strr1,str2 )
#print('Test hamdist :',res)


#ad=[['C', 'C', 'T', 'T', 'G', 'G', 't', 't', 'G', 'A'], ['C', 'C', 'T', 'T', 'G', 'G', 't', 't', 'T', 'T'], ['A', 'A', 'T', 'T', 'G', 'G', 't', 't', 'G', 'A']]
#motif = 'ccttggttga'
#resTotalDist = totalDistance(motif, ad,10 )
#print('Test TotalDistance :',resTotalDist) """

def MedianStringSearch(allkmers, sequences, k):
    """
    Implement l'algorithme MedianStringSearch
    entrée allkmers: liste de K-mers valides
    entrée sequences: matrice de dimension txn contenant les séquences
    entrée k: taille du motif 
    sortie bestMotif: le motif que minimise les distances
    sortie bestDistance: la distance minimal
    sortie motifDist: un dictionnaire contenant les motifs et leurs distances
    """
    bestDistance = 9999999999999
    motifDist = {}
    bestMotif = allkmers[0]

    for kmer in allkmers:
        motifDist[kmer] = totalDistance(kmer,sequences,k)
        if motifDist[kmer] < bestDistance:
            bestMotif=kmer
            bestDistance=motifDist[kmer]

    motifDist = dict(sorted(motifDist.items(), key=lambda item: item[1]))
    
    return bestMotif, bestDistance, motifDist


motif2= generateRandomSequence(k,False)
resTotalDist2 = totalDistance(motif2,adn,k)
print('\nMotif2 Test : ',motif2)
print(adn)


print('Test TotalDistance 2.0:',resTotalDist2)
print(MedianStringSearch(kmersValid, adn, k))



Motif2 Test :  CAAGGGGC
[['T', 'g', 'a', 't', 't', 'a', 'a', 'g', 'c', 'G'], ['g', 'a', 't', 't', 'a', 'c', 'g', 'c', 'C', 'C'], ['g', 't', 't', 't', 't', 't', 'g', 'c', 'G', 'G'], ['g', 'a', 't', 't', 'a', 't', 'a', 'c', 'C', 'T'], ['c', 'a', 't', 't', 'a', 't', 't', 'c', 'A', 'C'], ['C', 'T', 'g', 'a', 't', 't', 'a', 't', 'c', 'c'], ['G', 't', 'a', 't', 't', 'a', 'c', 'g', 'c', 'G'], ['C', 'G', 'c', 'a', 't', 't', 'a', 't', 'g', 'g'], ['g', 'a', 't', 't', 'a', 't', 'g', 'g', 'G', 'T'], ['g', 'a', 't', 'a', 'a', 't', 'g', 'c', 'G', 'A']]
Test TotalDistance 2.0: 51
('GATTATGC', 14, {'GATTATGC': 14, 'CATTATGC': 19, 'GATTACGC': 19, 'GATTAAGC': 20, 'GATTATAC': 20, 'GATTATCC': 20, 'GATTATGG': 20, 'GATTATTC': 20, 'TATTATGC': 20, 'AATTATGC': 21, 'GATTAGGC': 21, 'GATAATGC': 22, 'GATTATGA': 22, 'GATTATGT': 22, 'ATTATGCG': 23, 'GATCATGC': 23, 'GATGATGC': 23, 'GATTCTGC': 23, 'GATTGTGC': 23, 'GCTTATGC': 23, 'GGTTATGC': 23, 'CATTACGC': 24, 'GAATATGC': 24, 'GACTATGC': 24, 'GAGTATGC': 24, 'GATTACGG

In [76]:
#Appliquez l'algoritme MedianStringSearch sur les sequences du brin complementaire

def reverseComplement(seq):
    seq_dict = {'A':'T','T':'A','G':'C','C':'G'}
    return "".join([seq_dict[base] for base in reversed(seq)])

"""
seq='ATGC'
print(seq)
seq = reverseComplement(seq)
print(seq)"""

seqRev =[reverseComplement((''.join(e)).upper()) for e in adn ]
#print(adn)
#print(seqRev)

resMedString = MedianStringSearch(kmersValid,seqRev,k)
print(resMedString)

('GCATAATC', 14, {'GCATAATC': 14, 'GCATAATG': 19, 'GCGTAATC': 19, 'CCATAATC': 20, 'GAATAATC': 20, 'GCATAATA': 20, 'GCTTAATC': 20, 'GGATAATC': 20, 'GTATAATC': 20, 'GCATAATT': 21, 'GCCTAATC': 21, 'ACATAATC': 22, 'GCATTATC': 22, 'TCATAATC': 22, 'CGCATAAT': 23, 'GCACAATC': 23, 'GCAGAATC': 23, 'GCATAACC': 23, 'GCATAAGC': 23, 'GCATCATC': 23, 'GCATGATC': 23, 'CCGTAATC': 24, 'GCATACTC': 24, 'GCATAGTC': 24, 'GCATATTC': 24, 'GCGTAATA': 24, 'GCGTAATG': 24, 'CCATAATA': 25, 'CCATAATG': 25, 'GAATAATG': 25, 'GAGTAATC': 25, 'GCTTAATG': 25, 'GGATAATG': 25, 'GTATAATG': 25, 'GTGTAATC': 25, 'AGCATAAT': 26, 'CAATAATC': 26, 'CCTTAATC': 26, 'CGAATAAT': 26, 'CGATAATC': 26, 'CGCTTAAT': 26, 'CTATAATC': 26, 'GAATAATA': 26, 'GATTAATC': 26, 'GCCTAATG': 26, 'GCGTAATT': 26, 'GCTTAATA': 26, 'GGATAATA': 26, 'GGCATAAT': 26, 'GGTTAATC': 26, 'GTATAATA': 26, 'ACATAATG': 27, 'ACGTAATC': 27, 'CCATAATT': 27, 'CCGTAATA': 27, 'CGCGTAAT': 27, 'CGTATAAT': 27, 'GAATAATT': 27, 'GACTAATC': 27, 'GCATTATG': 27, 'GCCTAATA': 27, 'GCGTT

## Partie B : Recherche de motifs variable sur vos données

Chercher sur le fichier "Sequence_by_Peaks_G*.fasta" contiens les regions de peak trouvé par ChipSeq, qui contient probablement le Facteur de Transcription que nous cherchons. Apliquer l'alogorithm Median Sting pour chercher les motifs dans votre fichier "Sequence_by_Peaks_G*. Il faut bien evidement enlever les motifs peu complexe.

In [110]:
import time
import math

k=10
feq=10
top= 50
genome = "Sequence_by_Peaks_1.fasta"

def readFasta(fastaFileName):
    """
    Read a fasta file
    entrée fastaFileName: nom du fichier fasta
    sortie sequences: liste contenant toutes les sequences du fichier
    """
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())

    return sequence

def reversecomplList(seq):
    res=[reverseComplement(sq) for sq in seq]
    return res

def searchMotifs(k:int, sequences:list):
    """
    Cherche les motifs de taille k dans un ensemble de séquences
    entrée k : taille du motif
    entrée séquences : liste de sequences
    sortie motifs: liste de motif de taille k
    """
    motifs  = [];
    
    for i in range(0,len(sequences)):
        sequence=sequences[i]
        j=0
        while(j+k<len(sequence)):
            if sequence[j:j+k] not in motifs:
                motifs.append(sequence[j:j+k])
            j+=1
    
    return motifs

sequences = readFasta(genome)
brincompl = reversecomplList(sequences)
fusionListebr1et2 = sequences + brincompl

motifs = searchMotifs(k,fusionListebr1et2)
motifs = removeLowComplexe(motifs,3)
#motifs = removeLowComplexe(motifs,math.floor(k/2))

#print(MedianStringSearch(["TTTGCTCA", "CTCATTGG","CTGATTGG","TCCGCGGA","TCCGTGGA","CAGCAAAA","CGGATCCG"], motifs, k))
print(MedianStringSearch(["CCTTNAGGGT"], motifs, k))
#print(MedianStringSearch(["TGCACCC"], motifs, k))

('CCTTNAGGGT', 210838, {'CCTTNAGGGT': 210838})


************BRIN 1 SEULEMENT OUPS :

Si nb=4 pr removeLowComplex :
Motif de taille k=7 : ('TGCACCC', 55099, {'TGCACCC': 55099})
Motifs de taille 8 :('CAGCAAAA', 104249, {'CAGCAAAA': 104249, 'TTTGCTCA': 104820, 'CTGATTGG': 106123, 'CTCATTGG': 106142, 'TCCGTGGA': 107203, 'TCCGCGGA': 108492, 'CGGATCCG': 108850})
Motif de taille 10 :('CCTTNAGGGT', 171259, {'CCTTNAGGGT': 171259})


Si nb=3 pr removeLowComplex :
Motif de taille k=7 : ('TGCACCC', 43523, {'TGCACCC': 43523})
Motifs de taille 8 : ('CAGCAAAA', 75850, {'CAGCAAAA': 75850, 'TTTGCTCA': 76034, 'CTGATTGG': 77075, 'CTCATTGG': 77092, 'TCCGTGGA': 77561, 'TCCGCGGA': 78442, 'CGGATCCG': 78768})
Motif de taille 10 : ('CCTTNAGGGT', 107749, {'CCTTNAGGGT': 107749})

Si nb=2
Motif de taille 7 : ('TGCACCC', 9918, {'TGCACCC': 9918})
Motif de taille 10 :('CCTTNAGGGT', 11556, {'CCTTNAGGGT': 11556})
Motifs de taille 8 : ('TTTGCTCA', 13151, {'TTTGCTCA': 13151, 'CAGCAAAA': 13293, 'CTCATTGG': 13376, 'CTGATTGG': 13384, 'TCCGTGGA': 13395, 'TCCGCGGA': 13532, 'CGGATCCG': 13599})



<font color='blue'>
**************** BRIN1+BRIN2 ************

Si nb=4 pr removeLowComplex :
Motif de taille 7 :('TGCACCC', 67765, {'TGCACCC': 67765})
Motifs de taille 8 : ('CAGCAAAA', 167278, {'CAGCAAAA': 167278, 'TTTGCTCA': 167388, 'CTCATTGG': 169043, 'CTGATTGG': 169058, 'TCCGTGGA': 170780, 'TCCGCGGA': 172611, 'CGGATCCG': 172741})
Motif de taille 10 :('CCTTNAGGGT', 334110, {'CCTTNAGGGT': 334110})

Si nb=3 pr removeLowComplex :
Motif de taille 7 :('TGCACCC', 54266, {'TGCACCC': 54266})
Motifs de taille 8 :('CAGCAAAA', 123531, {'CAGCAAAA': 123531, 'TTTGCTCA': 123556, 'CTGATTGG': 124886, 'CTCATTGG': 124913, 'TCCGTGGA': 125779, 'TCCGCGGA': 127055, 'CGGATCCG': 127281})
Motif de taille 10 : ('CCTTNAGGGT', 210838, {'CCTTNAGGGT': 210838})

**************  CONCLUSION

Aft1 (TGCACCC) est donc le motif avec la plus petite TotalDistance qu'on retrouve peu importe la valeur de nb que nous fixons pour la suppression des motifs peu complex, c'est-à-dire avec le moins de variation et qui est donc très conservé. Il est donc potentiellement le motif que nous recherchons. 
    
Durée d'éxécutuion : Dépend des entrées
Compléxité : 
    
</font>

In [111]:
motifsFiltre = removeLowComplexe(sequences, 4)
bestMotif1, bestDistance1, motifDist1 = MedianStringSearch(kmersValid, sequences[0:3], k)
bestMotif2, bestDistance2, motifDist2 = MedianStringSearch(kmersValid, motifsFiltre, k)
print (bestMotif1,bestDistance1)
print (bestMotif2,bestDistance2)

AACAACAG 0
AACAACAG 0
